In [1]:
import pandas as pd
import yfinance as yf
import openpyxl
import json
import os
from dataclasses import dataclass
import matplotlib as plt
from datetime import datetime

In [2]:
@dataclass
class GraphPoint:
    index: int
    high: float
    low: float
    close: float
    volume: float
    date: str

In [3]:
@dataclass
class FilteredSymbols:
    latestPrice: float
    latestVolume: int
    fiftyDMA: float
    twoHundredDMA: float
    symbol: str
    recoveryPercentage: float

In [4]:
def shortListed500CrStocks(df, rowName: str, marketCapInCrs: float):
    return df[df[rowName] >= marketCapInCrs]

In [5]:
df = pd.read_excel('MCAP28032024.xlsx')
df = df.iloc[:-2, :]
df['Symbol'] = df['Symbol'] + '.NS'
df['Market capitalization as on March 28, 2024\n(In Cr.)']
filtered_df = shortListed500CrStocks(df,'Market capitalization as on March 28, 2024\n(In Cr.)', 500.0 )
filtered_df

,Sr. No.,Symbol,Company Name,"Market capitalization as on March 28, 2024\n(In lakhs)","Market capitalization as on March 28, 2024\n(In Cr.)"
0,1.0,RELIANCE.NS,Reliance Industries Limited,201056022.448876,2.010560e+06
1,2.0,TCS.NS,Tata Consultancy Services Limited,140247926.460234,1.402479e+06
2,3.0,HDFCBANK.NS,HDFC Bank Limited,109991524.325625,1.099915e+06
3,4.0,ICICIBANK.NS,ICICI Bank Limited,76765684.662095,7.676568e+05
4,5.0,BHARTIARTL.NS,Bharti Airtel Limited,69478399.828022,6.947840e+05
...,...,...,...,...,...
1412,1413.0,DONEAR.NS,Donear Industries Limited,50622,5.062200e+02
1413,1414.0,CAREERP.NS,Career Point Limited,50467.212786,5.046721e+02
1414,1415.0,INTLCONV.NS,International Conveyors Limited,50386.305,5.038630e+02
1415,1416.0,SHREEPUSHK.NS,Shree Pushkar Chemicals & Fertilisers Limited,50206.0845,5.020608e+02


In [6]:
filtered_tickers_symbol_list = filtered_df['Symbol'].astype(str).tolist()
filtered_tickers_symbol = ' '.join(filtered_tickers_symbol_list)

In [7]:
def getAllTimeHighPriceAndItsDateInLastVariableDays(df, prevNumberOfDays):
    # print(df)
    if(df.shape[0] < prevNumberOfDays):
        high_filtered_df = df['High']
    else:
        high_filtered_df = df.iloc[-prevNumberOfDays:, :]['High']
    # high_filtered_df.plot()
    high_filtered_df_high_index = high_filtered_df.idxmax() 
    return GraphPoint(high_filtered_df_high_index, high_filtered_df.max(), df.loc[high_filtered_df_high_index,'Low'], df.loc[high_filtered_df_high_index,'Close'], df.loc[high_filtered_df_high_index,'Volume'], df.loc[high_filtered_df_high_index,'Date'])

In [8]:
def getLowestPointAfterIndex(index: int, df):
    df_slice = df.iloc[index:, :]['Low']
    df_slice_low_point_index = df_slice.idxmin()
    return GraphPoint(df_slice_low_point_index,  df.loc[df_slice_low_point_index, 'High'], df_slice.min() , df.loc[df_slice_low_point_index, 'Close'], df.loc[df_slice_low_point_index, 'Volume'], df.loc[df_slice_low_point_index, 'Date'])

In [9]:
def getLatestClosePoint(symbol_max_history):
    return  GraphPoint(
        symbol_max_history.shape[0] - 1,
        symbol_max_history.iloc[-1]['High'],
        symbol_max_history.iloc[-1]['Low'],
        symbol_max_history.iloc[-1]['Close'],
        symbol_max_history.iloc[-1]['Volume'],
        symbol_max_history.iloc[-1]['Date'])

In [10]:
tickers = yf.Tickers(filtered_tickers_symbol)
sbin_ticker = tickers.tickers['RELIANCE.NS']

sbin_history = sbin_ticker.history(period="max")
sbin_history = sbin_history.reset_index(drop=False)
    
all_time_high_price = sbin_history['High'].max() 
highest_point_in_last_variable_days = getAllTimeHighPriceAndItsDateInLastVariableDays(sbin_history, 125)
print(highest_point_in_last_variable_days)

lowest_point_after_hitting_high = getLowestPointAfterIndex(highest_point_in_last_variable_days.index, sbin_history)
print(lowest_point_after_hitting_high)

GraphPoint(index=7159, high=3197.0, low=3096.0, close=3177.25, volume=6134855, date=Timestamp('2024-07-05 00:00:00+0530', tz='Asia/Kolkata'))
GraphPoint(index=7159, high=3197.0, low=3096.0, close=3177.25, volume=6134855, date=Timestamp('2024-07-05 00:00:00+0530', tz='Asia/Kolkata'))


In [11]:
def currValueGt(currVol: float, valueToCompareWith: float):
    return currVol > valueToCompareWith

def currValuePercentageChangeGt(currVol: float, valueToCompareWith: float, thresholdChange: float):
    return ((currVol/valueToCompareWith) > thresholdChange)

In [12]:
def getRecoveryPercentage(high_point: GraphPoint, lowest_piont_after_high: GraphPoint, latest_close_point: GraphPoint):
    return (latest_close_point.close - lowest_piont_after_high.low)/(high_point.high - lowest_piont_after_high.low)

In [13]:
numberOfDays = {"1d" : 1, "5d" : 5, "1mo" : 30 , "3mo" : 90, "6mo" : 180 , "1y": 360 , "2y": 720, "5y" : 1800 , "10y": 3600, "max": "max"}

In [14]:
def calculateAverageVolumeinLastXDays(numOfDays: str, tickerInfo):
    ticker_history_for_x_days = tickerInfo.history(period=numOfDays)
    return ticker_history_for_x_days['Volume'].sum() / numberOfDays[numOfDays]
    

In [15]:
latest_close_point = GraphPoint(sbin_history.shape[0] - 1, sbin_history.iloc[-1]['High'], sbin_history.iloc[-1]['Low'], sbin_history.iloc[-1]['Close'], sbin_history.iloc[-1]['Volume'], sbin_history.iloc[-1]['Date'])
getRecoveryPercentage(highest_point_in_last_variable_days, lowest_point_after_hitting_high, latest_close_point)
# currValueGt(0.9, getRecoveryPercentage(highest_point_in_last_variable_days, lowest_point_after_hitting_high, latest_close_point))

0.8044554455445545

In [16]:
def getLastXDaysVolumeAverage(numOfDays, ticker: yf.Ticker):
    if(numOfDays=="10d"):
        return ticker.info['averageVolume10days']
    else:
        return calculateAverageVolumeinLastXDays(numOfDays, ticker)

In [17]:
def getLastXDaysPriceAverage(numOfDays, tickerInfo: yf.Ticker):
    if(numOfDays=="10d"):
        return tickerInfo.info['averageVolume10days']
    else:
        return calculateAverageVolumeinLastXDays(numOfDays, tickerInfo)

In [18]:
# fifty_day_average_price = sbin_ticker.info['fiftyDayAverage']
# two_hundred_day_average_price = sbin_ticker.info['twoHundredDayAverage']

In [19]:
# getLastXDaysVolumeAverage("1d", sbin_ticker)

In [20]:
tickers = yf.Tickers(filtered_tickers_symbol)
print(filtered_tickers_symbol)
symbols_filtered = []
for symbol in filtered_tickers_symbol_list:
    print(f'\n [{datetime.now().time()}]')
    try:
        symbol_ticker = tickers.tickers[symbol]
        symbol_max_history = symbol_ticker.history(period="max")
        symbol_max_history = symbol_max_history.reset_index(drop=False)
        all_time_high = symbol_max_history['High'].max()
        fifty_day_average_price = symbol_ticker.info['fiftyDayAverage']
        two_hundred_day_average_price = symbol_ticker.info['twoHundredDayAverage']
        highest_point_in_last_variable_days = getAllTimeHighPriceAndItsDateInLastVariableDays(symbol_max_history, 125)
        lowest_point_after_hitting_high = getLowestPointAfterIndex(highest_point_in_last_variable_days.index, symbol_max_history)
        latest_close_point = getLatestClosePoint(symbol_max_history)
        recovery_percentage = getRecoveryPercentage(highest_point_in_last_variable_days, lowest_point_after_hitting_high, latest_close_point)
        last_5d_vol_average = getLastXDaysVolumeAverage("5d", symbol_ticker)
        last_1mo_vol_average = getLastXDaysVolumeAverage("1mo", symbol_ticker)
        current_volume = symbol_ticker.info['volume']
        print(f'{symbol} evaluated')
        if(
            currValueGt(recovery_percentage, 0.9) 
            and currValuePercentageChangeGt(current_volume, last_5d_vol_average, 2.0) 
            and currValueGt(current_volume, last_1mo_vol_average) 
            and currValueGt(latest_close_point.close, fifty_day_average_price) 
            and currValueGt(latest_close_point.close, two_hundred_day_average_price)):
            print(f'recovery_percentage : {recovery_percentage}\ncurr_volume : {current_volume}\nlast_5d_vol_average : {last_5d_vol_average}\nlast_1mo_vol_average : {last_1mo_vol_average}\nlatest_close_point : {latest_close_point.close}\nfifty_day_average_price : {fifty_day_average_price}\ntwo_hundred_day_average_price : {two_hundred_day_average_price}')
            print(f'----------------------------------------\n{symbol} found worthy to look into :)\n----------------------------------------')
            symbols_filtered.append(FilteredSymbols(latest_close_point.close, latest_close_point.volume, fifty_day_average_price, two_hundred_day_average_price, symbol, recovery_percentage))
    except:
        print(f'Data not found in yFinance for : {symbol}')
        continue
print(symbols_filtered)

RELIANCE.NS TCS.NS HDFCBANK.NS ICICIBANK.NS BHARTIARTL.NS SBIN.NS INFY.NS LICI.NS ITC.NS HINDUNILVR.NS LT.NS BAJFINANCE.NS HCLTECH.NS MARUTI.NS SUNPHARMA.NS ADANIENT.NS KOTAKBANK.NS TITAN.NS ONGC.NS TATAMOTORS.NS NTPC.NS AXISBANK.NS DMART.NS ADANIGREEN.NS ADANIPORTS.NS ULTRACEMCO.NS ASIANPAINT.NS COALINDIA.NS BAJAJFINSV.NS BAJAJ-AUTO.NS POWERGRID.NS NESTLEIND.NS WIPRO.NS M&M.NS IOC.NS JIOFIN.NS HAL.NS DLF.NS ADANIPOWER.NS JSWSTEEL.NS TATASTEEL.NS SIEMENS.NS IRFC.NS VBL.NS ZOMATO.NS PIDILITIND.NS GRASIM.NS SBILIFE.NS BEL.NS LTIM.NS TRENT.NS PNB.NS INDIGO.NS BANKBARODA.NS HDFCLIFE.NS ABB.NS BPCL.NS PFC.NS GODREJCP.NS TATAPOWER.NS HINDALCO.NS HINDZINC.NS TECHM.NS AMBUJACEM.NS INDUSINDBK.NS CIPLA.NS GAIL.NS RECLTD.NS BRITANNIA.NS UNIONBANK.NS ADANIENSOL.NS IOB.NS LODHA.NS EICHERMOT.NS CANBK.NS TATACONSUM.NS DRREDDY.NS TVSMOTOR.NS ZYDUSLIFE.NS ATGL.NS VEDL.NS CHOLAFIN.NS HAVELLS.NS HEROMOTOCO.NS DABUR.NS SHREECEM.NS MANKIND.NS BAJAJHLDNG.NS DIVISLAB.NS APOLLOHOSP.NS NHPC.NS SHRIRAMFIN.NS BO

$L&TFH.NS: possibly delisted; No timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/L&TFH.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=L%26TFH.NS&crumb=%2FiBp5L9X1WZ


Data not found in yFinance for : L&TFH.NS

 [22:15:52.519919]
UNOMINDA.NS evaluated

 [22:15:53.014709]
PAGEIND.NS evaluated

 [22:15:53.489317]
MRPL.NS evaluated

 [22:15:53.996760]
AUBANK.NS evaluated

 [22:15:54.392048]
MAZDOCK.NS evaluated

 [22:15:54.774016]
HUDCO.NS evaluated

 [22:15:55.163985]
GUJGASLTD.NS evaluated

 [22:15:55.578534]
NIACL.NS evaluated

 [22:15:55.980601]
CRISIL.NS evaluated

 [22:15:56.546591]
AIAENG.NS evaluated

 [22:15:57.001237]
FEDERALBNK.NS evaluated

 [22:15:57.533822]
IREDA.NS evaluated

 [22:15:57.866676]
VOLTAS.NS evaluated

 [22:15:58.459025]
DALBHARAT.NS evaluated

 [22:15:58.827820]
POONAWALLA.NS evaluated

 [22:15:59.247760]
MEDANTA.NS evaluated

 [22:15:59.580652]
IRB.NS evaluated

 [22:16:00.246104]
3MINDIA.NS evaluated

 [22:16:00.810235]
MFSL.NS evaluated

 [22:16:01.407557]
M&MFIN.NS evaluated

 [22:16:01.975875]
UPL.NS evaluated

 [22:16:02.544431]
HONAUT.NS evaluated

 [22:16:03.031832]
BSE.NS evaluated

 [22:16:03.441422]
FLUOROCHEM.NS 

$UJJIVAN.NS: possibly delisted; No price data found  (period=1mo)
$UJJIVAN.NS: possibly delisted; No price data found  (period=1mo)


$UJJIVAN.NS: possibly delisted; No price data found  (period=1mo)
UJJIVAN.NS evaluated
$UJJIVAN.NS: possibly delisted; No price data found  (period=1mo)
recovery_percentage : 0.9045215033645972
curr_volume : 2975180
last_10d_vol_average : 1835855
last_1mo_vol_average : 0.0
latest_close_point : 591.0
fifty_day_average_price : 530.027
two_hundred_day_average_price : 540.664
----------------------------------------
UJJIVAN.NS found worthy to look into :)
----------------------------------------

 [22:19:07.816787]
ITDCEM.NS evaluated

 [22:19:08.359743]
CYIENTDLM.NS evaluated

 [22:19:08.685479]
EPL.NS evaluated

 [22:19:09.267146]
SUPRAJIT.NS evaluated

 [22:19:09.778376]
SUNTECK.NS evaluated

 [22:19:10.226771]
HEMIPROP.NS evaluated

 [22:19:10.572966]
MOIL.NS evaluated

 [22:19:11.016792]
TIMETECHNO.NS evaluated

 [22:19:11.514523]
ASTRAMICRO.NS evaluated

 [22:19:11.995886]
TRIL.NS evaluated

 [22:19:12.464572]
WONDERLA.NS evaluated

 [22:19:12.853357]
ASKAUTOLTD.NS evaluated

 [22:19

...

Data not found in yFinance for : ACCENTMIC.NS

 [22:25:39.609289]
PTL.NS evaluated

 [22:25:40.201508]
INDOBORAX.NS evaluated

 [22:25:40.594046]
MOLDTECH.NS evaluated

 [22:25:41.090779]
MAZDA.NS evaluated

 [22:25:41.543493]
MINDTECK.NS evaluated

 [22:25:42.004577]


$COOLCAPS.NS: possibly delisted; No timezone found


Data not found in yFinance for : COOLCAPS.NS

 [22:25:43.089393]


$ALLETEC.NS: possibly delisted; No timezone found


Data not found in yFinance for : ALLETEC.NS

 [22:25:44.578883]
20MICRONS.NS evaluated

 [22:25:45.141958]
VIKASECO.NS evaluated

 [22:25:45.776705]
ORIENTBELL.NS evaluated

 [22:25:46.363402]
DONEAR.NS evaluated

 [22:25:47.041941]
CAREERP.NS evaluated

 [22:25:47.550987]
INTLCONV.NS evaluated

 [22:25:47.967180]
SHREEPUSHK.NS evaluated

 [22:25:48.450225]
PRITIKAUTO.NS evaluated
[FilteredSymbols(latestPrice=288.20001220703125, latestVolume=39781962, fiftyDMA=272.925, twoHundredDMA=241.357, symbol='ONGC.NS', recoveryPercentage=0.9320943649596313), FilteredSymbols(latestPrice=1162.449951171875, latestVolume=2734011, fiftyDMA=1050.496, twoHundredDMA=839.9585, symbol='ZYDUSLIFE.NS', recoveryPercentage=0.9578967393625086), FilteredSymbols(latestPrice=316.3999938964844, latestVolume=37284773, fiftyDMA=294.316, twoHundredDMA=221.333, symbol='BHEL.NS', recoveryPercentage=0.9380395501005189), FilteredSymbols(latestPrice=585.4000244140625, latestVolume=1773765, fiftyDMA=537.434, twoHundredDMA=

In [ ]:
[FilteredSymbols(latestPrice=288.20001220703125, latestVolume=39781962, fiftyDMA=272.925, twoHundredDMA=241.357, symbol='ONGC.NS', recoveryPercentage=0.9320943649596313), FilteredSymbols(latestPrice=1162.449951171875, latestVolume=2734011, fiftyDMA=1050.496, twoHundredDMA=839.9585, symbol='ZYDUSLIFE.NS', recoveryPercentage=0.9578967393625086), FilteredSymbols(latestPrice=316.3999938964844, latestVolume=37284773, fiftyDMA=294.316, twoHundredDMA=221.333, symbol='BHEL.NS', recoveryPercentage=0.9380395501005189), FilteredSymbols(latestPrice=585.4000244140625, latestVolume=1773765, fiftyDMA=537.434, twoHundredDMA=554.21326, symbol='STARHEALTH.NS', recoveryPercentage=0.9315265709694707), FilteredSymbols(latestPrice=612.1500244140625, latestVolume=637156, fiftyDMA=538.317, twoHundredDMA=489.855, symbol='CIEINDIA.NS', recoveryPercentage=0.9046405537203083), FilteredSymbols(latestPrice=922.0, latestVolume=466700, fiftyDMA=665.943, twoHundredDMA=441.95325, symbol='AEGISCHEM.NS', recoveryPercentage=0.9238579152014623), FilteredSymbols(latestPrice=848.7000122070312, latestVolume=569235, fiftyDMA=716.779, twoHundredDMA=628.698, symbol='SPLPETRO.NS', recoveryPercentage=0.9057354557641829), FilteredSymbols(latestPrice=1441.800048828125, latestVolume=430631, fiftyDMA=1317.554, twoHundredDMA=1171.8324, symbol='INOXINDIA.NS', recoveryPercentage=0.9137765558188764), FilteredSymbols(latestPrice=7926.64990234375, latestVolume=35603, fiftyDMA=7123.48, twoHundredDMA=5365.375, symbol='IWEL.NS', recoveryPercentage=0.9593081565487908), FilteredSymbols(latestPrice=187.99000549316406, latestVolume=17455116, fiftyDMA=161.4084, twoHundredDMA=150.65535, symbol='MARKSANS.NS', recoveryPercentage=0.9500992274520421), FilteredSymbols(latestPrice=591.0, latestVolume=2975180, fiftyDMA=530.027, twoHundredDMA=540.664, symbol='UJJIVAN.NS', recoveryPercentage=0.9045215033645972), FilteredSymbols(latestPrice=1609.800048828125, latestVolume=210441, fiftyDMA=1253.108, twoHundredDMA=1072.2946, symbol='AURIONPRO.NS', recoveryPercentage=0.9969868875628141), FilteredSymbols(latestPrice=924.5499877929688, latestVolume=2316870, fiftyDMA=714.48, twoHundredDMA=822.85425, symbol='AGI.NS', recoveryPercentage=0.9389089133522728), FilteredSymbols(latestPrice=364.6000061035156, latestVolume=345592, fiftyDMA=297.762, twoHundredDMA=288.48624, symbol='DBCORP.NS', recoveryPercentage=0.9367273988733352), FilteredSymbols(latestPrice=1790.199951171875, latestVolume=133347, fiftyDMA=1444.758, twoHundredDMA=1135.778, symbol='DHANUKA.NS', recoveryPercentage=0.9221291161758473), FilteredSymbols(latestPrice=554.4500122070312, latestVolume=392716, fiftyDMA=500.507, twoHundredDMA=497.438, symbol='AARTIDRUGS.NS', recoveryPercentage=0.9010588785309136), FilteredSymbols(latestPrice=216.35000610351562, latestVolume=4754589, fiftyDMA=195.636, twoHundredDMA=182.16376, symbol='HINDOILEXP.NS', recoveryPercentage=0.9039498898052382), FilteredSymbols(latestPrice=127.43000030517578, latestVolume=3065454, fiftyDMA=101.7516, twoHundredDMA=95.51365, symbol='SUVEN.NS', recoveryPercentage=0.9659066321220517), FilteredSymbols(latestPrice=1406.300048828125, latestVolume=230822, fiftyDMA=969.103, twoHundredDMA=723.112, symbol='DREDGECORP.NS', recoveryPercentage=1.0), FilteredSymbols(latestPrice=300.32000732421875, latestVolume=1836114, fiftyDMA=222.8834, twoHundredDMA=199.9561, symbol='WALCHANNAG.NS', recoveryPercentage=1.0)]


In [29]:
# tickers.tickers['AEGISCHEM.NS'].info['']
getLastXDaysVolumeAverage('10d', tickers.tickers['AEGISCHEM.NS'])

1101857